In [ ]:
ALTER SESSION SET TIMEZONE = 'Europe/London';

In [1]:
import os
import requests
from azure.identity import ClientSecretCredential
import pandas as pd
from json import loads, dumps
from datetime import datetime, timedelta

try:
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()

    tenant_id = session.sql("SELECT prod.raw.ADO_DEFENDER_TENANT_ID()").collect()[0][0]

    client_id = session.sql("SELECT prod.raw.ADO_DEFENDER_CLIENT_ID()").collect()[0][0]

    client_secret = session.sql("SELECT prod.raw.ADO_DEFENDER_CLIENT_SECRET()").collect()[0][0]

    subscription_id = session.sql("SELECT prod.raw.ADO_DEFENDER_SUBSCRIPTION_ID()").collect()[0][0]

    billing_account_id = session.sql("SELECT PROD.RAW.billing_account_id()").collect()[0][0]
    
except:
    from dotenv import load_dotenv
    load_dotenv()
    tenant_id = os.getenv('tenant_id')
    client_id = os.getenv('client_id')
    client_secret = os.getenv('client_secret')
    subscription_id = os.getenv('subscription_id')
    billing_account_id = os.getenv('billing_account_id')


In [2]:

credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")
headers = {"Authorization": f"Bearer {token.token}", "Content-Type": "application/json"}
def get_all_usage_details():
    all_data = []
    url = f"https://management.azure.com/providers/Microsoft.Billing/billingAccounts/{billing_account_id}/providers/Microsoft.Consumption/usageDetails"

    # Ostatni miesiąc
    end_date = datetime.now()
    start_date = end_date - timedelta(days=1)

    params = {
        "api-version": "2023-05-01",
        # "metric": "AmortizedCost",
        # "$top": 1000,
        "startDate": start_date.strftime("%Y-%m-%d"),
        "endDate": end_date.strftime("%Y-%m-%d")
    }
    
    # params = {
    #     "api-version": "2023-05-01",
    #     # "key": "usagedetailsamortized",
    #     # "$top": 1000,
    #     "startDate": '2025-07-01' ,
    #     "endDate": '2025-07-31'
    # }

    while url:
        response = requests.get(url, headers=headers, params=params if '?' not in url else None)
        data = response.json()

        all_data.extend(data.get('value', []))
        url = data.get('nextLink')  # następna strona
        params = None  # nextLink już zawiera wszystkie parametry

        print(f"Pobrano {len(all_data)} rekordów...")

    return all_data
# Pobierz wszystkie dane
usage_data = get_all_usage_details()
df = pd.DataFrame(usage_data)
print(f"Łącznie rekordów: {len(df)}")

Pobrano 1000 rekordów...
Pobrano 2000 rekordów...
Pobrano 3000 rekordów...
Pobrano 4000 rekordów...
Pobrano 5000 rekordów...
Pobrano 6000 rekordów...
Pobrano 7000 rekordów...
Pobrano 8000 rekordów...
Pobrano 9000 rekordów...
Pobrano 10000 rekordów...
Pobrano 11000 rekordów...
Pobrano 12000 rekordów...
Pobrano 13000 rekordów...
Pobrano 14000 rekordów...
Pobrano 15000 rekordów...
Pobrano 16000 rekordów...
Pobrano 17000 rekordów...
Pobrano 18000 rekordów...
Pobrano 19000 rekordów...
Pobrano 20000 rekordów...
Pobrano 21000 rekordów...
Pobrano 22000 rekordów...
Pobrano 23000 rekordów...
Pobrano 24000 rekordów...
Pobrano 25000 rekordów...
Pobrano 26000 rekordów...
Pobrano 27000 rekordów...


KeyboardInterrupt: 

In [ ]:
df

In [ ]:

COSTS_FROM_PYTHON = 'COSTS_FROM_PYTHON'

session.write_pandas(df,
                         table_name=COSTS_FROM_PYTHON,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
create or replace table prod.raw.azure_cost_manager  as
select * from COSTS_FROM_PYTHON

In [ ]:

insert into prod.silver.azure_cost_manager
select 
parse_json("properties"):"additionalInfo" as additionalInfo,
parse_json("properties"):"benefitId" as benefitId,
parse_json("properties"):"benefitName" as benefitName,
parse_json("properties"):"billingAccountId" as billingAccountId,
parse_json("properties"):"billingAccountName" as billingAccountName,
parse_json("properties"):"billingCurrencyCode" as billingCurrencyCode,
parse_json("properties"):"billingPeriodEndDate" as billingPeriodEndDate,
parse_json("properties"):"billingPeriodStartDate" as billingPeriodStartDate,
parse_json("properties"):"billingProfileId" as billingProfileId,
parse_json("properties"):"billingProfileName" as billingProfileName,
parse_json("properties"):"chargeType" as chargeType,
parse_json("properties"):"consumedService" as consumedService,
parse_json("properties"):"costAllocationRuleName" as costAllocationRuleName,
parse_json("properties"):"costCenter" as costCenter,
parse_json("properties"):"costInBillingCurrency" as costInBillingCurrency,
parse_json("properties"):"costInPricingCurrency" as costInPricingCurrency,
parse_json("properties"):"costInUSD" as costInUSD,
parse_json("properties"):"customerName" as customerName,
parse_json("properties"):"customerTenantId" as customerTenantId,
parse_json("properties"):"date" as date,
parse_json("properties"):"effectivePrice" as effectivePrice,
NULLIF(parse_json("properties"):"exchangeRate"::string, '') as exchangeRate,
parse_json("properties"):"exchangeRateDate" as exchangeRateDate,
parse_json("properties"):"exchangeRatePricingToBilling" as exchangeRatePricingToBilling,
parse_json("properties"):"frequency" as frequency,
parse_json("properties"):"instanceName" as instanceName,
parse_json("properties"):"invoiceId" as invoiceId,
parse_json("properties"):"invoiceSectionId" as invoiceSectionId,
parse_json("properties"):"invoiceSectionName" as invoiceSectionName,
parse_json("properties"):"isAzureCreditEligible" as isAzureCreditEligible,
parse_json("properties"):"meterCategory" as meterCategory,
parse_json("properties"):"meterId" as meterId,
parse_json("properties"):"meterName" as meterName,
parse_json("properties"):"meterRegion" as meterRegion,
parse_json("properties"):"meterSubCategory" as meterSubCategory,
parse_json("properties"):"partnerEarnedCreditApplied" as partnerEarnedCreditApplied,
parse_json("properties"):"partnerEarnedCreditRate" as partnerEarnedCreditRate,
parse_json("properties"):"partnerName" as partnerName,
parse_json("properties"):"partnerTenantId" as partnerTenantId,
parse_json("properties"):"payGPrice" as payGPrice,
parse_json("properties"):"paygCostInBillingCurrency" as paygCostInBillingCurrency,
parse_json("properties"):"paygCostInUSD" as paygCostInUSD,
parse_json("properties"):"previousInvoiceId" as previousInvoiceId,
parse_json("properties"):"pricingCurrencyCode" as pricingCurrencyCode,
parse_json("properties"):"pricingModel" as pricingModel,
parse_json("properties"):"product" as product,
parse_json("properties"):"productIdentifier" as productIdentifier,
parse_json("properties"):"productOrderId" as productOrderId,
parse_json("properties"):"productOrderName" as productOrderName,
parse_json("properties"):"provider" as provider,
parse_json("properties"):"publisherId" as publisherId,
parse_json("properties"):"publisherName" as publisherName,
parse_json("properties"):"publisherType" as publisherType,
parse_json("properties"):"quantity" as quantity,
parse_json("properties"):"resellerMpnId" as resellerMpnId,
parse_json("properties"):"resellerName" as resellerName,
parse_json("properties"):"reservationId" as reservationId,
parse_json("properties"):"reservationName" as reservationName,
parse_json("properties"):"resourceGroup" as resourceGroup,
parse_json("properties"):"resourceLocation" as resourceLocation,
parse_json("properties"):"resourceLocationNormalized" as resourceLocationNormalized,
parse_json("properties"):"serviceFamily" as serviceFamily,
parse_json("properties"):"serviceInfo1" as serviceInfo1,
parse_json("properties"):"serviceInfo2" as serviceInfo2,
parse_json("properties"):"servicePeriodEndDate" as servicePeriodEndDate,
parse_json("properties"):"servicePeriodStartDate" as servicePeriodStartDate,
parse_json("properties"):"subscriptionGuid" as subscriptionGuid,
parse_json("properties"):"subscriptionName" as subscriptionName,
parse_json("properties"):"term" as term,
parse_json("properties"):"unitOfMeasure" as unitOfMeasure,
parse_json("properties"):"unitPrice" as unitPrice

from prod.raw.azure_cost_manager

In [ ]:
-- select distinct "type"  from COSTS_FROM_PYTHON

In [ ]:
-- select 
-- parse_json("properties"):"date"::date as date, 
-- sum(parse_json("properties"):"costInBillingCurrency")::int as costInBillingCurrency
-- from COSTS_FROM_PYTHON
-- group by date
-- -- where date

In [ ]:
-- select 
-- parse_json("properties"):"date" as date, 
-- sum(parse_json("properties"):"costInBillingCurrency")::int as costInBillingCurrency
-- from prod.raw.azure_cost_manager
-- group by date
-- -- where date

In [ ]:
-- select 
-- parse_json("properties"):"additionalInfo" as additionalInfo,
-- parse_json("properties"):"benefitId" as benefitId,
-- parse_json("properties"):"benefitName" as benefitName,
-- parse_json("properties"):"billingAccountId" as billingAccountId,
-- parse_json("properties"):"billingAccountName" as billingAccountName,
-- parse_json("properties"):"billingCurrencyCode" as billingCurrencyCode,
-- parse_json("properties"):"billingPeriodEndDate" as billingPeriodEndDate,
-- parse_json("properties"):"billingPeriodStartDate" as billingPeriodStartDate,
-- parse_json("properties"):"billingProfileId" as billingProfileId,
-- parse_json("properties"):"billingProfileName" as billingProfileName,
-- parse_json("properties"):"chargeType" as chargeType,
-- parse_json("properties"):"consumedService" as consumedService,
-- parse_json("properties"):"costAllocationRuleName" as costAllocationRuleName,
-- parse_json("properties"):"costCenter" as costCenter,
-- parse_json("properties"):"costInBillingCurrency" as costInBillingCurrency,
-- parse_json("properties"):"costInPricingCurrency" as costInPricingCurrency,
-- parse_json("properties"):"costInUSD" as costInUSD,
-- parse_json("properties"):"customerName" as customerName,
-- parse_json("properties"):"customerTenantId" as customerTenantId,
-- parse_json("properties"):"date" as date,
-- parse_json("properties"):"effectivePrice" as effectivePrice,
-- NULLIF(parse_json("properties"):"exchangeRate"::string, '') as exchangeRate,
-- parse_json("properties"):"exchangeRateDate" as exchangeRateDate,
-- parse_json("properties"):"exchangeRatePricingToBilling" as exchangeRatePricingToBilling,
-- parse_json("properties"):"frequency" as frequency,
-- parse_json("properties"):"instanceName" as instanceName,
-- parse_json("properties"):"invoiceId" as invoiceId,
-- parse_json("properties"):"invoiceSectionId" as invoiceSectionId,
-- parse_json("properties"):"invoiceSectionName" as invoiceSectionName,
-- parse_json("properties"):"isAzureCreditEligible" as isAzureCreditEligible,
-- parse_json("properties"):"meterCategory" as meterCategory,
-- parse_json("properties"):"meterId" as meterId,
-- parse_json("properties"):"meterName" as meterName,
-- parse_json("properties"):"meterRegion" as meterRegion,
-- parse_json("properties"):"meterSubCategory" as meterSubCategory,
-- parse_json("properties"):"partnerEarnedCreditApplied" as partnerEarnedCreditApplied,
-- parse_json("properties"):"partnerEarnedCreditRate" as partnerEarnedCreditRate,
-- parse_json("properties"):"partnerName" as partnerName,
-- parse_json("properties"):"partnerTenantId" as partnerTenantId,
-- parse_json("properties"):"payGPrice" as payGPrice,
-- parse_json("properties"):"paygCostInBillingCurrency" as paygCostInBillingCurrency,
-- parse_json("properties"):"paygCostInUSD" as paygCostInUSD,
-- parse_json("properties"):"previousInvoiceId" as previousInvoiceId,
-- parse_json("properties"):"pricingCurrencyCode" as pricingCurrencyCode,
-- parse_json("properties"):"pricingModel" as pricingModel,
-- parse_json("properties"):"product" as product,
-- parse_json("properties"):"productIdentifier" as productIdentifier,
-- parse_json("properties"):"productOrderId" as productOrderId,
-- parse_json("properties"):"productOrderName" as productOrderName,
-- parse_json("properties"):"provider" as provider,
-- parse_json("properties"):"publisherId" as publisherId,
-- parse_json("properties"):"publisherName" as publisherName,
-- parse_json("properties"):"publisherType" as publisherType,
-- parse_json("properties"):"quantity" as quantity,
-- parse_json("properties"):"resellerMpnId" as resellerMpnId,
-- parse_json("properties"):"resellerName" as resellerName,
-- parse_json("properties"):"reservationId" as reservationId,
-- parse_json("properties"):"reservationName" as reservationName,
-- parse_json("properties"):"resourceGroup" as resourceGroup,
-- parse_json("properties"):"resourceLocation" as resourceLocation,
-- parse_json("properties"):"resourceLocationNormalized" as resourceLocationNormalized,
-- parse_json("properties"):"serviceFamily" as serviceFamily,
-- parse_json("properties"):"serviceInfo1" as serviceInfo1,
-- parse_json("properties"):"serviceInfo2" as serviceInfo2,
-- parse_json("properties"):"servicePeriodEndDate" as servicePeriodEndDate,
-- parse_json("properties"):"servicePeriodStartDate" as servicePeriodStartDate,
-- parse_json("properties"):"subscriptionGuid" as subscriptionGuid,
-- parse_json("properties"):"subscriptionName" as subscriptionName,
-- parse_json("properties"):"term" as term,
-- parse_json("properties"):"unitOfMeasure" as unitOfMeasure,
-- parse_json("properties"):"unitPrice" as unitPrice

-- from COSTS_FROM_PYTHON

In [ ]:
-- select 
-- parse_json("properties"):"date"::date as date ,
-- count(parse_json("properties"):"date"::date) as date_count
-- -- * 

-- from COSTS_FROM_PYTHON
-- group by date
-- -- where servicePeriodStartDate <> '2025-07-01'

In [ ]:
-- select 
-- parse_json("properties"):"servicePeriodStartDate"::date as servicePeriodStartDate ,
-- count(parse_json("properties"):"servicePeriodStartDate"::date) as servicePeriodStartDate_count
-- -- * 

-- from COSTS_FROM_PYTHON
-- group by servicePeriodStartDate
-- -- where servicePeriodStartDate <> '2025-07-01'

In [ ]:
-- select 
-- parse_json("properties"):"servicePeriodStartDate"::date as servicePeriodStartDate ,
-- count(parse_json("properties"):"servicePeriodStartDate"::date) as servicePeriodStartDate_count
-- -- * 

-- from COSTS_FROM_PYTHON
-- group by servicePeriodStartDate
-- -- where servicePeriodStartDate <> '2025-07-01'

In [ ]:
-- select 
-- servicePeriodStartDate ,
-- count(servicePeriodStartDate) as servicePeriodStartDate_count
-- -- * 

-- from prod.silver.azure_cost_manager
-- group by servicePeriodStartDate
-- -- where servicePeriodStartDate <> '2025-07-01'

In [ ]:
-- select 
-- date ,
-- count(date) as date_count
-- -- * 

-- from prod.silver.azure_cost_manager
-- -- where date < '2025-07-01'
-- group by date


In [ ]:
-- select count(*) from prod.raw.azure_cost_manager 

In [ ]:
-- insert into prod.raw.azure_cost_manager 
-- select * from COSTS_FROM_PYTHON

In [ ]:
-- select count(*) from prod.raw.azure_cost_manager 

In [ ]:
-- select top 10 * from prod.raw.azure_cost_manager

In [ ]:
-- create or replace table prod.silver.azure_cost_manager as
-- select
-- parse_json("properties"):"additionalInfo"::variant as additionalInfo,
-- parse_json("properties"):"benefitId"::string as benefitId,
-- parse_json("properties"):"benefitName"::string as benefitName,
-- parse_json("properties"):"billingAccountId"::string as billingAccountId,
-- parse_json("properties"):"billingAccountName"::string as billingAccountName,
-- parse_json("properties"):"billingCurrencyCode"::string as billingCurrencyCode,
-- parse_json("properties"):"billingPeriodEndDate"::date as billingPeriodEndDate,
-- parse_json("properties"):"billingPeriodStartDate"::date as billingPeriodStartDate,
-- parse_json("properties"):"billingProfileId"::string as billingProfileId,
-- parse_json("properties"):"billingProfileName"::string as billingProfileName,
-- parse_json("properties"):"chargeType"::string as chargeType,
-- parse_json("properties"):"consumedService"::string as consumedService,
-- parse_json("properties"):"costAllocationRuleName"::string as costAllocationRuleName,
-- parse_json("properties"):"costCenter"::string as costCenter,
-- parse_json("properties"):"costInBillingCurrency"::float as costInBillingCurrency,
-- parse_json("properties"):"costInPricingCurrency"::float as costInPricingCurrency,
-- parse_json("properties"):"costInUSD"::float as costInUSD,
-- parse_json("properties"):"customerName"::string as customerName,
-- parse_json("properties"):"customerTenantId"::string as customerTenantId,
-- parse_json("properties"):"date"::date as date,
-- parse_json("properties"):"effectivePrice"::float as effectivePrice,
-- NULLIF(parse_json("properties"):"exchangeRate"::string, '')::float as exchangeRate,
-- parse_json("properties"):"exchangeRateDate"::date as exchangeRateDate,
-- parse_json("properties"):"exchangeRatePricingToBilling"::float as exchangeRatePricingToBilling,
-- parse_json("properties"):"frequency"::string as frequency,
-- parse_json("properties"):"instanceName"::string as instanceName,
-- parse_json("properties"):"invoiceId"::string as invoiceId,
-- parse_json("properties"):"invoiceSectionId"::string as invoiceSectionId,
-- parse_json("properties"):"invoiceSectionName"::string as invoiceSectionName,
-- parse_json("properties"):"isAzureCreditEligible"::string as isAzureCreditEligible,
-- parse_json("properties"):"meterCategory"::string as meterCategory,
-- parse_json("properties"):"meterId"::string as meterId,
-- parse_json("properties"):"meterName"::string as meterName,
-- parse_json("properties"):"meterRegion"::string as meterRegion,
-- parse_json("properties"):"meterSubCategory"::string as meterSubCategory,
-- parse_json("properties"):"partnerEarnedCreditApplied"::string as partnerEarnedCreditApplied,
-- parse_json("properties"):"partnerEarnedCreditRate"::string as partnerEarnedCreditRate,
-- parse_json("properties"):"partnerName"::string as partnerName,
-- parse_json("properties"):"partnerTenantId"::string as partnerTenantId,
-- parse_json("properties"):"payGPrice"::float as payGPrice,
-- parse_json("properties"):"paygCostInBillingCurrency"::float as paygCostInBillingCurrency,
-- parse_json("properties"):"paygCostInUSD"::float as paygCostInUSD,
-- parse_json("properties"):"previousInvoiceId"::string as previousInvoiceId,
-- parse_json("properties"):"pricingCurrencyCode"::string as pricingCurrencyCode,
-- parse_json("properties"):"pricingModel"::string as pricingModel,
-- parse_json("properties"):"product"::string as product,
-- parse_json("properties"):"productIdentifier"::string as productIdentifier,
-- parse_json("properties"):"productOrderId"::string as productOrderId,
-- parse_json("properties"):"productOrderName"::string as productOrderName,
-- parse_json("properties"):"provider"::string as provider,
-- parse_json("properties"):"publisherId"::string as publisherId,
-- parse_json("properties"):"publisherName"::string as publisherName,
-- parse_json("properties"):"publisherType"::string as publisherType,
-- parse_json("properties"):"quantity"::int as quantity,
-- parse_json("properties"):"resellerMpnId"::string as resellerMpnId,
-- parse_json("properties"):"resellerName"::string as resellerName,
-- parse_json("properties"):"reservationId"::string as reservationId,
-- parse_json("properties"):"reservationName"::string as reservationName,
-- parse_json("properties"):"resourceGroup"::string as resourceGroup,
-- parse_json("properties"):"resourceLocation"::string as resourceLocation,
-- parse_json("properties"):"resourceLocationNormalized"::string as resourceLocationNormalized,
-- parse_json("properties"):"serviceFamily"::string as serviceFamily,
-- parse_json("properties"):"serviceInfo1"::string as serviceInfo1,
-- parse_json("properties"):"serviceInfo2"::string as serviceInfo2,
-- parse_json("properties"):"servicePeriodEndDate"::date as servicePeriodEndDate,
-- parse_json("properties"):"servicePeriodStartDate"::date as servicePeriodStartDate,
-- parse_json("properties"):"subscriptionGuid"::string as subscriptionGuid,
-- parse_json("properties"):"subscriptionName"::string as subscriptionName,
-- parse_json("properties"):"term"::string as term,
-- parse_json("properties"):"unitOfMeasure"::string as unitOfMeasure,
-- parse_json("properties"):"unitPrice"::float as unitPrice

-- from prod.raw.azure_cost_manager

In [ ]:
select count(*) from prod.silver.azure_cost_manager

In [ ]:
-- select distinct subscriptionname from prod.silver.azure_cost_manager

In [ ]:
-- -- po staremu
-- SELECT month(date) as month_date, subscriptionname, SUM(costInBillingCurrency) 
-- FROM prod.silver.azure_cost_manager
-- GROUP BY month_date, subscriptionname

In [ ]:
-- select * from prod.silver.azure_cost_manager

In [ ]:
-- SELECT month(date) as month_date, subscriptionname, SUM(costInBillingCurrency) 
-- FROM prod.silver.azure_cost_manager
-- WHERE subscriptionname not LIKE '%Develop%'
-- GROUP BY month_date, subscriptionname

In [ ]:
-- SELECT month(date) as month_date, subscriptionname, SUM(costInBillingCurrency) 
-- FROM prod.silver.azure_cost_manager
-- WHERE subscriptionname LIKE '%Develop%'
-- GROUP BY month_date, subscriptionname